In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import pandas as pd
import scipy.sparse as sp

from DataParser import DataParser
from CFItemKNN import CFItemKNN
import SimpleEvaluator as evaluator

from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

In [2]:
parser = DataParser()

In [3]:
URM_all = parser.get_URM_all()
URM_train, URM_test = split_train_in_two_percentage_global_sample(URM_all, train_percentage=0.85)

In [4]:
MAP_per_k = []
x_tick_t= []

In [ ]:
x_tick = [10, 50, 100, 500, 900, 1000]
recommender = CFItemKNN(URM_train)
for topK in x_tick:
    recommender.fit(shrink=10, topK=topK)
    result_dict= evaluator.evaluator(recommender,URM, cutoff=10)
    MAP_per_k.append(result_dict["MAP"])
    x_tick_t.append(topK)

In [ ]:
plt.plot(x_tick_t, MAP_per_k)
plt.ylabel('MAP')
plt.xlabel('TopK')
plt.show()

In [ ]:
x_tick = [0, 10, 30, 50, 100]
MAP_per_shrinkage = []

recommender = CFItemKNN(URM_train)

for shrink in x_tick:
    
    recommender.fit(shrink=shrink, topK=100)
    result_dict= evaluator.evaluator(recommender,URM_valid, cutoff=10)    
    MAP_per_shrinkage.append(result_dict["MAP"])

In [ ]:
plt.plot(x_tick, MAP_per_shrinkage)
plt.ylabel('MAP')
plt.xlabel('Shrinkage')
plt.show()

In [5]:
URM_train, URM_test = split_train_in_two_percentage_global_sample(URM_all, train_percentage=0.85)
rec2 = CFItemKNN(URM_train)
rec2.fit(shrink=356, topK=967)
res_dict = evaluator.evaluator(rec2, URM_test, cutoff=10)

Similarity column 25975 ( 100 % ), 2363.83 column/sec, elapsed time 0.18 min


In [6]:
res_dict

{'precision': 0.02670272435897397,
 'recall': 0.1027467565263828,
 'MAP': 0.05024608771237305}

### Predictions

In [7]:
import re

In [20]:
out_userid = np.array([])
out_itemlist = np.array([])

target_data = pd.read_csv('data/data_target_users_test.csv',
                          header=0,
                          names=["user_id"],
                          dtype={"user_id":np.int32})

for user_id in target_data.user_id.unique():
    out_userid = np.append(out_userid, user_id)
    str_=re.sub(' +', ' ', np.array_str(rec2.recommend(user_id,at=10)))[1:-1]
    if str_[0]==' ':
        str_=str_[1:]
    out_itemlist = np.append(out_itemlist, str_)
#re.sub('[','', re.sub('[ ','', 
out_dataframe = pd.DataFrame(data={'user_id':out_userid, 'item_list':out_itemlist})
out_dataframe = out_dataframe.astype({'user_id': 'int32'})

In [21]:
out_itemlist

array(['1447 5085 637 4927 22850 13657 20451 18390 909 24780',
       '19089 12409 23600 19709 8894 16630 19480 3165 17257 20095',
       '25693 3164 1876 18790 16901 6662 25491 5717 8374 4842', ...,
       '24093 15764 16857 12348 21962 11153 10574 3924 16823 2685',
       '19010 19480 19089 7313 11687 8894 10269 9339 19524 22511',
       '2523 6861 10752 6828 16522 10471 14971 17897 14904 20348'],
      dtype='<U59')